In [1]:
from astropy.cosmology import Planck13
import pandas as pd
import numpy as np
import disperse
from tqdm import tqdm
import json
%matplotlib inline
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'disperse'

In [ ]:
cosmo = Planck13

H0 = cosmo.H0.value
Om = cosmo.Om0
Ol = 0.69288
Ok = 0.0

sph2cart = 'dist'
in_cart_coords = True

BOARD = 'smooth'
ANGLE = 30

In [ ]:
coma_DR7 = pd.read_csv('coma_DR7_filtered_RaDecZ.csv')
coma_DR12 = pd.read_csv('coma_DR12_filtered_RaDecZ.csv')
rosat = pd.read_csv('rosat_filtered_RaDecZ.csv')
sdss_opt_gr_full = pd.read_csv('sdss_opt_gr_filtered_RaDecZ.csv')

In [ ]:
# N_GAL >= 15

In [ ]:
min_n_gals = 15

In [ ]:
sdss_opt_gr = sdss_opt_gr_full[sdss_opt_gr_full['n_gal'] >= min_n_gals]
sdss_opt_gr.reset_index(drop=True, inplace=True)

In [ ]:
DPS = disperse.Disperse3D(
    coma_DR12, sdss_opt_gr, '_disperse_03/bin/',
    H0, Om, Ol, Ok,
    sph2cart_f=sph2cart
)
DPS.count_cart_coords()

CX = DPS.galaxies['CX']
CY = DPS.galaxies['CY']
CZ = DPS.galaxies['CZ']

CX_int = (CX.min(), CX.max())
CY_int = (CY.min(), CY.max())
CZ_int = (CZ.min(), CZ.max())

np.random.seed(0)

random_clusters_num = 5
random_clusters = []
for i in range(random_clusters_num):
    cx = np.random.uniform(CX_int[0], CX_int[1], DPS.clusters.shape[0])
    cy = np.random.uniform(CY_int[0], CY_int[1], DPS.clusters.shape[0])
    cz = np.random.uniform(CZ_int[0], CZ_int[1], DPS.clusters.shape[0])
    df = pd.DataFrame()
    df = df.assign(CX=cx)
    df = df.assign(CY=cy)
    df = df.assign(CZ=cz)
    df = df.assign(R200=DPS.clusters['R200'])
    random_clusters.append(df)

In [ ]:
random_clusters[0]

In [ ]:
metrics = {}

In [ ]:
RAD_COEFS = list(np.arange(1, 20, 1.0))
metrics['mode'] = 'MPC'
metrics['COEFS'] = RAD_COEFS
metrics['min_n_gals'] = min_n_gals
metrics['total_clusters'] = sdss_opt_gr.shape[0]

In [ ]:
for SIGMA in list(np.arange(3, 5.5, 1.0)):
    metrics[SIGMA] = {}
    for SMOOTH in range(0, 3):
        metrics[SIGMA][SMOOTH] = {}
        DPS = disperse.Disperse3D(
            coma_DR12, sdss_opt_gr, '_disperse_03/bin/',
            H0, Om, Ol, Ok,
            sph2cart_f=sph2cart
        )
        DPS.count_cart_coords()
        DPS.apply_disperse(SIGMA, SMOOTH, BOARD, ANGLE, in_cart_coords)
        true_inter = []
        true_fils_inter = []
        for COEF in tqdm(RAD_COEFS):
            cl_conn, fils_conn = DPS.count_conn(
                [COEF] * sdss_opt_gr.shape[0]
            )
            true_inter.append(sum(list(map(lambda x: int(x>0), cl_conn))))
            true_fils_inter.append(sum(list(map(lambda x: int(x>0), fils_conn))))
        true_inter = np.array(true_inter)
        true_fils_inter = np.array(true_fils_inter)
        
        false_inter = []
        false_fils_inter = []
        for i in tqdm(range(random_clusters_num)):
            false_inter.append([])
            false_fils_inter.append([])
            for COEF in RAD_COEFS:
                cl_conn, fils_conn = DPS.count_conn(
                    [COEF] * sdss_opt_gr.shape[0],
#                     sdss_opt_gr['R200'] * COEF,
                    random_clusters[i]
                )
                false_inter[i].append(sum(list(map(lambda x: int(x>0), cl_conn))))
                false_fils_inter[i].append(sum(list(map(lambda x: int(x>0), fils_conn))))
                
        false_inter = np.array(false_inter).mean(0)
        false_fils_inter = np.array(false_fils_inter).mean(0)
        
        diff = true_inter - false_inter
        fils_diff = true_fils_inter - false_fils_inter
        
        true_inter = [int(e) for e in true_inter]
        false_inter = [float(e) for e in false_inter]
        diff = [float(e) for e in diff]
        true_fils_inter = [int(e) for e in true_fils_inter]
        false_fils_inter = [int(e) for e in false_fils_inter]
        fils_diff = [int(e) for e in fils_diff]
        
        metrics[SIGMA][SMOOTH]['true'] = list(true_inter)
        metrics[SIGMA][SMOOTH]['false'] = list(false_inter)
        metrics[SIGMA][SMOOTH]['diff'] = list(diff)
        metrics[SIGMA][SMOOTH]['fils_num'] = len(DPS.fils)
        metrics[SIGMA][SMOOTH]['true_fils_inter'] = list(true_fils_inter)
        metrics[SIGMA][SMOOTH]['false_fils_inter'] = list(false_fils_inter)
        metrics[SIGMA][SMOOTH]['fils_diff'] = list(fils_diff)

In [ ]:
metrics

In [ ]:
with open("metrics_temp.json", "w") as f:
    json.dump(metrics, f)